In [1]:
!pip install transformers accelerate bitsandbytes

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import pandas as pd

In [4]:
from transformers import AutoModelForCausalLM, LlamaTokenizer
import torch

# Load the LLaMA tokenizer and model
tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", device_map="auto", load_in_4bit=True)

# Set the model to evaluation mode
model.eval()



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Lla

In [38]:
def load_column_descriptions(csv_file):
    df = pd.read_csv(csv_file)
    return df

# Step 2: Generate a detailed description using the LLaMA model
def generate_table_description(column_descriptions, model, col_name, col_desc):
    unique_rows = column_descriptions.drop_duplicates(subset=[col_name, col_desc])

    # Initialize the table description
    table_description = "The table contains the following columns:\n\n"

    # Iterate over the unique rows to build the description
    for _, row in unique_rows.iterrows():
        column_name = row[col_name]
        description = row[col_desc]
        table_description += f"- {column_name}: {description}\n"

    # Generate a detailed description using the LLaMA model
    prompt = f"Given the following table structure, provide a short but complete description of the table based on column name descriptions provided in the structure:\n\n{table_description}\n\nDescription:"
    model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")
    # Generate a response from the model
    with torch.no_grad():
        outputs = model.generate(**model_inputs)

    # Decode the generated text
    desc = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return desc

In [39]:
column_descriptions = load_column_descriptions("/content/filtered_col_sim.csv")
detailed_description = generate_table_description(column_descriptions, model, "SAP_COL_NAME", "SAP_COL_DESCRIPTION")

In [40]:
print(detailed_description)

Given the following table structure, provide a short but complete description of the table based on column name descriptions provided in the structure:

The table contains the following columns:

- PLFAZ: Project planned start date
- PLSEZ: Project planned finish date
- VGPLF: Planning method for project basic dates
- BERBU: Authorization key for project budget
- PSPID: Project Definition
- PSPNR: Project definition (internal)


Description:
This table stores information about projects, including their planned start and finish dates, planning method, budget authorization key, and project definition. The table provides a comprehensive overview of project planning and execution, enabling effective monitoring and control of project progress.


In [41]:
column_descriptions = load_column_descriptions("/content/filtered_col_sim.csv")
detailed_description = generate_table_description(column_descriptions, model, "ORACLE_COL_NAME", "ORACLE_COL_DESCRIPTION")

In [42]:
print(detailed_description)

Given the following table structure, provide a short but complete description of the table based on column name descriptions provided in the structure:

The table contains the following columns:

- END_DATE_ACTIVE: The last effective date for the project party
- GRANT_ID: Identifier of the grant that the user has on the project.
- PROJECT_ID: The identifier of the project to which the objects belong
- START_DATE_ACTIVE: The first effective date for the project party


Description:
This table stores information about the active status of project parties, including the last and first effective dates, and the grant ID and project ID. The table provides a way to keep track of the status of project parties over time and the relationships between grants and projects.


In [57]:
column_descriptions = load_column_descriptions("/content/test.csv")
detailed_description = generate_table_description(column_descriptions, model, "SAP_COL_NAME", "SAP_COL_DESCRIPTION")

In [58]:
print(detailed_description)

Given the following table structure, provide a short but complete description of the table based on column name descriptions provided in the structure:

The table contains the following columns:

- BELNR: Accounting Document Number
- BLDAT: Document Date in Document
- BUKRS: Company Code
- BUZEI: Number of Line Item Within Accounting Document
- GJAHR: Fiscal Year
- LIFNR: Account Number of Vendor or Creditor
- MANDT: Client
- SHKZG: Debit/Credit Indicator
- WAERS: Currency Key
- WRBTR: Amount in document currency
- XBLNR: Reference Document Number


Description:
This table stores information about accounting documents, such as vendors, creditors, and customers. The table contains information about the accounting document number, date, company code, number of line items, fiscal year, account number, client, debit/credit indicator, currency, amount, and reference document number.


In [52]:
df = pd.read_csv("/content/filtered_dfAP.csv")

In [53]:
df

,ORACLE_TABLE_ID,ORACLE_TABLE_NAME,ORACLE_TABLE_DESCRIPTION,oracle_emb_description,key,SAP_TABLE_NAME,SAP_TABLE_DESCRIPTION,sap_emb_description,cos_sim
0,50650.0,AP_TERMS_LINES,Detail information about payment terms,"[0.0038840326, 0.063422956, -0.02268669, -0.02...",1,BSIP,Index for Vendor Validation of Double Documents,"[-0.040152825, 0.0445492, 0.016624687, 0.03346...",0.101477
1,50650.0,AP_TERMS_LINES,Detail information about payment terms,"[0.0038840326, 0.063422956, -0.02268669, -0.02...",1,CDHDR,Change document header,"[-0.040619086, 0.17945038, 0.038660202, 0.0356...",0.042087
2,50650.0,AP_TERMS_LINES,Detail information about payment terms,"[0.0038840326, 0.063422956, -0.02268669, -0.02...",1,TOA01,Link table 1,"[-0.007778221, -0.05693968, -0.071177974, 0.04...",0.078598
3,50650.0,AP_TERMS_LINES,Detail information about payment terms,"[0.0038840326, 0.063422956, -0.02268669, -0.02...",1,VIBDINFRA,Infrastructure,"[0.00697069, 0.02622691, -0.008817527, -0.0184...",0.131527
4,50650.0,AP_TERMS_LINES,Detail information about payment terms,"[0.0038840326, 0.063422956, -0.02268669, -0.02...",1,ADR12,FTP and URL (Business Address Services),"[-0.0007563168, -0.013080988, 0.001799077, 0.0...",0.133388
...,...,...,...,...,...,...,...,...,...
350,50650.0,AP_TERMS_LINES,Detail information about payment terms,"[0.0038840326, 0.063422956, -0.02268669, -0.02...",1,CSSK,Cost center /cost element,"[-0.019444322, -0.007927906, -0.04686664, 0.04...",0.186953
351,50650.0,AP_TERMS_LINES,Detail information about payment terms,"[0.0038840326, 0.063422956, -0.02268669, -0.02...",1,EQUI,Equipment master data,"[-0.010385334, -0.0043849135, 0.002226889, -0....",0.150431
352,50650.0,AP_TERMS_LINES,Detail information about payment terms,"[0.0038840326, 0.063422956, -0.02268669, -0.02...",1,PRTE,Scheduling Data for Project Item,"[-0.0330936, 0.07665708, -0.0052175852, 0.0630...",0.049641
353,50650.0,AP_TERMS_LINES,Detail information about payment terms,"[0.0038840326, 0.063422956, -0.02268669, -0.02...",1,TCYTM,Transition matrix,"[-0.05299969, 0.002179376, -0.05610997, -0.022...",0.031430
